# Create sinc-sinc-sine probes to use in Asterix

The DM probe files are defined as (Matthews et al., 2017):

$$\phi_j(u,v) = sinc(f_1 u)~sinc(f_2 v)~sin(2\pi f_3 u+\theta_j)$$

where $u,v$ are the positions in the DM pupil plane and $f_i$ are the spatial frequencies for the probe region.

The in-pupil DM on THD2 (DM2) has a square array of 32 x 32 actuators, where only 27 x 27 are inside the pupil mask. I will be creating the probe files on a 32 x 32 array and then make sure they are within the 27 x 27 visible area. In a later step, I might have to shift the probes around so that they are unobscured by the Roman HLC Lyot stop.

When using single-actuator probes, the respective DM probe gets created like this:
```
Voltage_probe = np.zeros(DM_probe.number_act)
Voltage_probe[i] = amplitude
probephase[k] = DM_probe.voltage_to_phase(Voltage_probe)
```

In [ ]:
import os
from astropy.io import fits
from matplotlib.colors import TwoSlopeNorm
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np

from Asterix import main_THD, Asterix_root
from Asterix.utils import get_data_dir, read_parameter_file
from Asterix.optics import Pupil, Coronagraph, DeformableMirror, Testbed

zerocenter = TwoSlopeNorm(vcenter=0.)

# 1. Set up THD2 object

In [ ]:
# Instantiate a THD2 object
your_directory = Asterix_root
your_parameter_file_name = 'thd2_setups/roman_637nm.ini'
parameter_file_path = os.path.join(your_directory, your_parameter_file_name)

config = read_parameter_file(parameter_file_path)

data_dir = get_data_dir(config_in=config["Data_dir"])
model_local_dir = os.path.join(data_dir, "Model_local")

# Concatenate into the full testbed optical system
thd2 = main_THD.THD2(config, model_local_dir)
original_dm = thd2.dm2.DM_pushact

dm_ref = thd2.dm2.DM_pushact[518] + thd2.dm2.DM_pushact[519] + thd2.dm2.DM_pushact[780]
dm = np.sum(thd2.dm2.DM_pushact, axis=0)
pup = thd2.entrance_pupil.pup
ls = thd2.corono.lyot_pup.pup

In [ ]:
masked_ls = np.ma.masked_not_equal(ls, 1.)
#masked_ls = np.ma.masked_where(ls == 1, ls)

bbox = 50
plt.imshow(masked_ls[bbox:-bbox, bbox:-bbox])
plt.colorbar()

In [ ]:
# THD2 pupil files
thd2_ls = fits.getdata('/Users/ilaginja/repos/Asterix/Asterix/model/roman_lyot_thd2_500pix_center4pixels.fits').astype('bool')
thd2_pupil = fits.getdata('/Users/ilaginja/repos/Asterix/Asterix/model/roman_pup_thd2_500pix_center4pixels.fits')
print(thd2_ls.shape)

masked_thd2_ls = np.ma.masked_where(thd2_ls == 1, thd2_ls)

# 2. Probe construction in general

Playing around and learning how to do this.

With two cardinal sines and a sine.

In [ ]:
acts_across = 32
sinc_freq = 14   # cycles per DM
sine_freq = 10    # cycles per DM
theta = np.pi
amp_ptv = 20   # nm

xvals_sinc = (np.arange(acts_across) / acts_across - 0.5) * sinc_freq
xvals_sine = (np.arange(acts_across) / acts_across - 0.5) * 2 * np.pi * sine_freq

xx, yy = np.meshgrid(xvals_sinc, xvals_sinc)
xx_sine, yy_sine = np.meshgrid(xvals_sine, xvals_sine)
probe = amp_ptv * np.sinc(xx) * np.sinc(yy) * np.sin(xx_sine + theta)

probe_sincs = amp_ptv * np.sinc(xx) * np.sinc(yy)
probe_sine = amp_ptv * np.sin(xx_sine)

plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
plt.imshow(probe_sincs, origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.title('Two sincs')
plt.colorbar()
plt.subplot(1, 3, 2)
plt.imshow(probe_sine, origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.title('A sine')
plt.colorbar()
plt.subplot(1, 3, 3)
plt.imshow(probe, origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.title('Sinc-sinc-sine')
plt.colorbar()

### Have a look at the pure FT of the sinc-sinc-sine probe

In [ ]:
transform = np.fft.ifftshift(np.fft.fft2(np.fft.fftshift(probe)))

In [ ]:
imnorm = TwoSlopeNorm(vmin=-23, vcenter=0, vmax=20)

plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.imshow(np.real(transform), origin='lower', cmap='RdBu', norm=imnorm)
plt.title('Real part of probe transform')
plt.colorbar()
plt.subplot(2, 2, 2)
plt.imshow(np.imag(transform), origin='lower', cmap='RdBu')#, norm=imnorm)
plt.title('Imag part of probe transform')
plt.colorbar()
plt.subplot(2, 2, 3)
plt.imshow(np.abs(transform), origin='lower', cmap='Reds')#, norm=imnorm)
plt.title('Amplitude of probe transform')
plt.colorbar()
plt.subplot(2, 2, 4)
plt.imshow(np.angle(transform), origin='lower', cmap='RdBu')#, norm=imnorm)
plt.title('Phase of probe transform')
plt.colorbar()

### Probe position

Now create the same probe (rotate the sine if you wish to) but shift it to where they are the least blocked by the Roman HLC Lyot stop and spiders.

In [ ]:
act = 1 / 32
shift_x = 0 * act
shift_y = -10.5 * act

acts_across = 32
sinc_freq = 14   # cycles per DM
sine_freq = 10    # cycles per DM
theta = np.pi
amp_ptv = 20   # nm

xvals_sinc = (np.arange(acts_across) / acts_across - 0.5 - shift_x) * sinc_freq
yvals_sinc = (np.arange(acts_across) / acts_across - 0.5 - shift_y) * sinc_freq
xx, yy = np.meshgrid(xvals_sinc, yvals_sinc)

xvals_sine = (np.arange(acts_across) / acts_across - 0.5 - shift_x) * 2 * np.pi * sine_freq
yvals_sine = (np.arange(acts_across) / acts_across - 0.5 - shift_y) * 2 * np.pi * sine_freq
xx_sine, yy_sine = np.meshgrid(xvals_sine, yvals_sine)

probe = amp_ptv * np.sinc(xx) * np.sinc(yy) * np.sin(yy_sine + theta)

plt.figure(figsize=(10, 10))
plt.imshow(probe, origin='lower', cmap='RdBu', norm=zerocenter)
plt.title('Shifted sinc-sinc-sine')
plt.colorbar()

In [ ]:
probe_sincs = amp_ptv * np.sinc(xx) * np.sinc(yy)
probe_sine = amp_ptv * np.sin(xx_sine)

plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
plt.imshow(probe_sincs, origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.title('Two sincs')
plt.colorbar()
plt.subplot(1, 3, 2)
plt.imshow(probe_sine, origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.title('A sine')
plt.colorbar()
plt.subplot(1, 3, 3)
plt.imshow(probe, origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.title('Sinc-sinc-sine')
plt.colorbar()

### Probe phases

Create a set of probes with different phases to use in the PW estimation.

In [ ]:
act = 1 / 32
shift_x = 0 * act
shift_y = -10.5 * act

acts_across = 32
sinc_freq = 10   # cycles per DM
sine_freq = 14    # cycles per DM
# thetas = [np.pi, 2 * np.pi / 3, np.pi / 3]
thetas = [2 * np.pi, 5 * np.pi / 3, np.pi / 3]
amp_ptv = 1   # nm

xvals_sinc = (np.arange(acts_across) / acts_across - 0.5 - shift_x) * sinc_freq
yvals_sinc = (np.arange(acts_across) / acts_across - 0.5 - shift_y) * sinc_freq
xx, yy = np.meshgrid(xvals_sinc, yvals_sinc)

xvals_sine = (np.arange(acts_across) / acts_across - 0.5 - shift_x) * 2 * np.pi * sine_freq
yvals_sine = (np.arange(acts_across) / acts_across - 0.5 - shift_y) * 2 * np.pi * sine_freq
xx_sine, yy_sine = np.meshgrid(xvals_sine, yvals_sine)

probes = []
for theta in thetas:
    probe = amp_ptv * np.sinc(xx) * np.sinc(yy) * np.sin(yy_sine + theta)
    probes.append(probe)

plt.figure(figsize=(16, 24))
plt.subplot(3, 2, 1)
plt.imshow(probes[0], origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.colorbar()
plt.subplot(3, 2, 2)
plt.imshow(-probes[0], origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.colorbar()
plt.subplot(3, 2, 3)
plt.imshow(probes[1], origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.colorbar()
plt.subplot(3, 2, 4)
plt.imshow(-probes[1], origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.colorbar()
plt.subplot(3, 2, 5)
plt.imshow(probes[2], origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.colorbar()
plt.subplot(3, 2, 6)
plt.imshow(-probes[2], origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.colorbar()

In [ ]:
# If I wanted to save this to disk
# Note the 90 deg rotation necessary for Asterix
for i, probe in enumerate(probes):
    path = '/Users/ilaginja/Documents/LESIA/THD/Roman_probes/sinc_probes'
    fname = f'probe_{i}_sinc-freq{sinc_freq}_sine-freq{sine_freq}.fits'
    fits.writeto(os.path.join(path, fname), np.rot90(probe))

# 3. Probes for Asterix

### Probe phases with different x and y spatial frequencies in the sinc functions

The below is what I use to create probe files for Asterix.

In [ ]:
act = 1 / 32
shift_x = 0 * act
shift_y = -9 * act

acts_across = 32
sinc_freq1 = 10   # cycles per DM --> left-right in DH space and independent of sine --> ideally 2 * OWA + margin, but our DMs cannot do that
sinc_freq2 = 22   # cycles per DM --> ~OWA in lam/D + margin
sine_freq = 5    # cycles per DM --> ~ at the midpoint between IWA and OWA
#thetas = [2 * np.pi, 5 * np.pi / 3, 4 * np.pi / 3, np.pi, 2 * np.pi / 3, np.pi / 3]   # 6 values that sample 0 - 2pi evenly
# thetas = [2 * np.pi, 5 * np.pi / 3, np.pi / 3]   # original probes
thetas = [2 * np.pi, 4 * np.pi / 3, 2 * np.pi / 3]
amp_ptv = 1   # nm

xvals_sinc = (np.arange(acts_across) / acts_across - 0.5 - shift_x) * sinc_freq1
yvals_sinc = (np.arange(acts_across) / acts_across - 0.5 - shift_y) * sinc_freq2
xx, yy = np.meshgrid(xvals_sinc, yvals_sinc)

xvals_sine = (np.arange(acts_across) / acts_across - 0.5 - shift_x) * 2 * np.pi * sine_freq
yvals_sine = (np.arange(acts_across) / acts_across - 0.5 - shift_y) * 2 * np.pi * sine_freq
xx_sine, yy_sine = np.meshgrid(xvals_sine, yvals_sine)

probes = []
for theta in thetas:
    probe = amp_ptv * np.sinc(xx) * np.sinc(yy) * np.sin(xx_sine + theta)
#     probe.ravel()[518] = 1000   # Make actuator 518 stand out
#     probe.ravel()[519] = 1000
#     probe.ravel()[780] = 1000
    probes.append(np.rot90(probe))  # Rotation to make consistent with Asterix

    
plt.figure(figsize=(16, 24))

plt.subplot(3, 2, 1)
plt.imshow(probes[0], origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 2)
plt.imshow(-probes[0], origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 3)
plt.imshow(probes[1], origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 4)
plt.imshow(-probes[1], origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 5)
plt.imshow(probes[2], origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 6)
plt.imshow(-probes[2], origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.colorbar()

#plt.savefig('probes.pdf')

In [ ]:
probe_sincs = amp_ptv * np.sinc(xx) * np.sinc(yy)
probe_sine = amp_ptv * np.sin(xx_sine)

plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
plt.imshow(probe_sincs, origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.title('Two sincs')
plt.colorbar()
plt.subplot(1, 3, 2)
plt.imshow(probe_sine, origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.title('A sine')
plt.colorbar()
plt.subplot(1, 3, 3)
plt.imshow(probe, origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.title('Sinc-sinc-sine')
plt.colorbar()

## Saving for Asterix

!! **The above plots show the probes with respect to the DM map we use to refer to actuators. For Asterix, any DM map needs to be rotated by 90 degrees to be used correctly.** !!

In [ ]:
for i, probe in enumerate(probes):
    path = '/Users/ilaginja/Documents/LESIA/THD/Roman_probes/sinc_probes'
    fname = f'probe_{i}_sinc-freq1-{sinc_freq1}_sinc-freq2-{sinc_freq2}_sine-freq{sine_freq}_shiftx{shift_x}_shifty{shift_y}.fits'
    fits.writeto(os.path.join(path, fname), probe)

# 4. Creating different types of probes

... for Asterix.

## a) The SPHERE way

Adapted from Axel's SPHERE code.

In [ ]:
def create_sinc_probe(probe_index, central_act, act_matrix, IWA, OWA, amplitudePW):
    '''
    Create an orthoganal set of sinc functions

    Parameters
    ----------
    pupsize : integer
        Size of the pupil in pixel.
    probe_index : integer
        Probe index.
    central_act : integer
        Max of the sinc function corresponds to the position of central_act index actuator
    act_matrix : 3D array
        Cube of actuator influence functions.
    IWA : float
        Inner working angle. (Should be 0, did not try with >0)
    OWA : float
        Outer working angle. Should not be higher than nb of actuators across the pupil divided by 2
    amplitudePW : float
        Max amplitude of the theoretical sinc function / 37

    Returns
    -------
    applied_sinc : 2D array
        Probe shape.
    voltage_sinc : 1D array
        Voltage being applied to the DM to create the probe.

    '''
    pupsize = act_matrix.shape[-1]
    
    nb1 = OWA - IWA
    nb2 = OWA * 2
    delta_xy = (OWA + IWA) /2
    coord = np.linspace(-0.5, 0.5, pupsize)
    xx, yy = np.meshgrid(coord, coord)
    
    #Find actuator position that will correspond to sinc max
    on_act = np.argwhere( np.abs(act_matrix[central_act] ) == np.amax( np.abs(act_matrix[central_act]) ))[0]
    
    #Definition for the three sinc functions
    if probe_index == 0:
        theoretical_sinc = np.sinc(nb1 * xx) * np.sinc(nb2 * yy) * np.cos(2 * delta_xy * np.pi * xx + 2 * np.pi * (probe_index+1)/4)
    elif probe_index == 1:
        theoretical_sinc = np.sinc(nb2 * xx) * np.sinc(nb2 * yy)
    elif probe_index == 2:
        theoretical_sinc = np.sinc(nb2 * xx) * np.sinc(nb1 * yy) * np.cos(2 * delta_xy * np.pi * yy + 2 * np.pi * (probe_index+1)/4)
        
        
    #Set amplitude of the sinc in nm
    theoretical_sinc = theoretical_sinc * amplitudePW * 37
    
    #Move sinc to on_act position
    theoretical_sinc = np.roll(theoretical_sinc, int(on_act[0] - pupsize/2), axis = 0)
    theoretical_sinc = np.roll(theoretical_sinc, int(on_act[1] - pupsize/2)+1, axis = 1)
    
    #Project the sinc function on the DM act basis (1e2 is the regularisation term. 1e3 is too tight, 1e1 does not do anything)
    applied_sinc, voltage_sinc = fit_DM_surf(act_matrix, theoretical_sinc, 1e2)
    
    return applied_sinc, voltage_sinc

In [ ]:
act = 1 / 32

OWA = 10
IWA = 3

nb1 = OWA - IWA
nb2 = OWA * 2
delta_xy = (OWA + IWA) /2

acts_across = 32
amp_ptv = 1   # nm

# sinc_freq1 = OWA * 2   # cycles per DM --> left-right in DH space and independent of sine --> ideally 2 * OWA + margin, but our DMs cannot do that
# sinc_freq2 = OWA - IWA   # cycles per DM --> ~OWA in lam/D + margin
# sine_freq = (OWA + IWA) /2    # cycles per DM --> ~ at the midpoint between IWA and OWA

sinc_freq1 = 22   # cycles per DM --> left-right in DH space and independent of sine --> ideally 2 * OWA + margin, but our DMs cannot do that
sinc_freq2 = 10   # cycles per DM --> ~OWA in lam/D + margin
sine_freq = 5    # cycles per DM --> ~ at the midpoint between IWA and OWA

probes = []

# PROBE 1
shift_x = -5 * act
shift_y = -8 * act

xvals_sinc = (np.arange(acts_across) / acts_across - 0.5 - shift_x) * sinc_freq2
yvals_sinc = (np.arange(acts_across) / acts_across - 0.5 - shift_y) * sinc_freq1
xx, yy = np.meshgrid(xvals_sinc, yvals_sinc)

xvals_sine = (np.arange(acts_across) / acts_across - 0.5 - shift_x) * 2 * np.pi * sine_freq 
yvals_sine = (np.arange(acts_across) / acts_across - 0.5 - shift_y) * 2 * np.pi * sine_freq
xx_sine, yy_sine = np.meshgrid(xvals_sine, yvals_sine)

probe = amp_ptv * np.sinc(xx) * np.sinc(yy) * np.cos(xx_sine + np.pi/2)
probes.append(probe)

# PROBE 2
shift_x = -9 * act
shift_y = -0.8 * act

xvals_sinc = (np.arange(acts_across) / acts_across - 0.5 - shift_x) * sinc_freq1
yvals_sinc = (np.arange(acts_across) / acts_across - 0.5 - shift_y) * sinc_freq2
xx, yy = np.meshgrid(xvals_sinc, yvals_sinc)

probe = amp_ptv * np.sinc(xx) * np.sinc(yy)
probes.append(probe)

# PROBE 3
shift_x = 7 * act
shift_y = -5 * act

xvals_sinc = (np.arange(acts_across) / acts_across - 0.5 - shift_x) * sinc_freq1
yvals_sinc = (np.arange(acts_across) / acts_across - 0.5 - shift_y) * sinc_freq2
xx, yy = np.meshgrid(xvals_sinc, yvals_sinc)

xvals_sine = (np.arange(acts_across) / acts_across - 0.5 - shift_x) * 2 * np.pi * sine_freq
yvals_sine = (np.arange(acts_across) / acts_across - 0.5 - shift_y) * 2 * np.pi * sine_freq
xx_sine, yy_sine = np.meshgrid(xvals_sine, yvals_sine)

probe = amp_ptv * np.sinc(xx) * np.sinc(yy) * np.cos(yy_sine + 3 * np.pi / 2)
probes.append(np.rot90(probe))   # Rotation to make consistent with Asterix

zerocenter = TwoSlopeNorm(vcenter=0.)

plt.figure(figsize=(16, 24))
plt.subplot(3, 2, 1)
plt.imshow(probes[0], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()
plt.subplot(3, 2, 2)
plt.imshow(-probes[0], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()
plt.subplot(3, 2, 3)
plt.imshow(probes[1], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()
plt.subplot(3, 2, 4)
plt.imshow(-probes[1], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()
plt.subplot(3, 2, 5)
plt.imshow(probes[2], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()
plt.subplot(3, 2, 6)
plt.imshow(-probes[2], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

In [ ]:
probe_sincs = amp_ptv * np.sinc(xx) * np.sinc(yy)
probe_sine = amp_ptv * np.sin(xx_sine)

zerocenter = TwoSlopeNorm(vcenter=0.)

plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
plt.imshow(probe_sincs, origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.title('Two sincs')
plt.colorbar()
plt.subplot(1, 3, 2)
plt.imshow(probe_sine, origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.title('A sine')
plt.colorbar()
plt.subplot(1, 3, 3)
plt.imshow(probe, origin='lower', cmap='RdBu')#, norm=zerocenter)
plt.title('Sinc-sinc-sine')
plt.colorbar()

### Save for Asterix

In [ ]:
for i, probe in enumerate(probes):
    path = '/Users/ilaginja/Documents/LESIA/THD/Roman_probes/sinc_probes'
    fname = f'probe_{i}_sinc-freq1-{sinc_freq1}_sinc-freq2-{sinc_freq2}_sine-freq{sine_freq}_custom_shifts.fits'
    fits.writeto(os.path.join(path, fname), probe)

## b) Single-actuator probes

In [ ]:
act_probes = [297, 298, 266]
#act_probes = [518, 519, 551]
#act_probes = [506, 485, 362]

amp_ptv = 1
acts_across = 32

probes = []
for i in act_probes:
    act_command = np.zeros((acts_across, acts_across))
    act_command.ravel()[i] = amp_ptv
    probes.append(act_command)

zerocenter = TwoSlopeNorm(vcenter=0.)

plt.figure(figsize=(16, 24))

plt.subplot(3, 2, 1)
plt.imshow(probes[0], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 2)
plt.imshow(-probes[0], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 3)
plt.imshow(probes[1], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 4)
plt.imshow(-probes[1], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 5)
plt.imshow(probes[2], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 6)
plt.imshow(-probes[2], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

### Save for Asterix

In [ ]:
for i, probe in enumerate(probes):
    path = '/Users/ilaginja/Documents/LESIA/THD/Roman_probes/actuator_probes/2024-12-09_297-298-266'
    fname = f'probe_{i}.fits'
    fits.writeto(os.path.join(path, fname), probe)

## c) CGI probes

Or, adapted sinc-sinc-sine probes for THD2 with a Roman HLC.

In [ ]:
act = 1 / 32

shift_x_A = 7 * act
shift_y_A = -5 * act

shift_x_B = 7 * act
shift_y_B = -5 * act

shift_x_C = 7 * act
shift_y_C = -5 * act

acts_across = 32
sinc_freq1 = 12   # cycles per DM --> OWA-IWA
sinc_freq2 = 24   #  cycles per DM --> left-right in DH space and independent of sine --> ideally 2 * OWA + margin, but our DMs cannot do that   
sine_freq = 6     # cycles per DM --> (OWA + IWA) / 2
thetas = [2 * np.pi, 4 * np.pi / 3, 2 * np.pi / 3]
amp_ptv = 1   # nm

xvals_sinc_A = (np.arange(acts_across) / acts_across - 0.5 - shift_x_A) * sinc_freq1
yvals_sinc_A = (np.arange(acts_across) / acts_across - 0.5 - shift_y_A) * sinc_freq2
xx_A, yy_A = np.meshgrid(xvals_sinc_A, yvals_sinc_A)

xvals_sinc_B = (np.arange(acts_across) / acts_across - 0.5 - shift_x_B) * sinc_freq2
yvals_sinc_B = (np.arange(acts_across) / acts_across - 0.5 - shift_y_B) * sinc_freq2
xx_B, yy_B = np.meshgrid(xvals_sinc_B, yvals_sinc_B)

xvals_sinc_C = (np.arange(acts_across) / acts_across - 0.5 - shift_x_C) * sinc_freq2
yvals_sinc_C = (np.arange(acts_across) / acts_across - 0.5 - shift_y_C) * sinc_freq1
xx_C, yy_C = np.meshgrid(xvals_sinc_C, yvals_sinc_C)

xvals_sine = (np.arange(acts_across) / acts_across - 0.5 - 0) * 2 * np.pi * sine_freq
yvals_sine = (np.arange(acts_across) / acts_across - 0.5 - 0) * 2 * np.pi * sine_freq
xx_sine, yy_sine = np.meshgrid(xvals_sine, yvals_sine)

probes = []

probe1 = amp_ptv * np.sinc(xx_A) * np.sinc(yy_A) * np.sin(xx_sine + 1 * np.pi / 4)
probe2 = amp_ptv * np.sinc(xx_B) * np.sinc(yy_B) #* np.cos(yy_sine + 2 * np.pi / 3)
probe3 = amp_ptv * np.sinc(xx_C) * np.sinc(yy_C) * np.sin(yy_sine + 3 * np.pi / 4)

probes.append(np.rot90(probe1))  # Rotation to make consistent with Asterix
probes.append(np.rot90(probe2))
probes.append(np.rot90(probe3))

zerocenter = TwoSlopeNorm(vcenter=0.)
    
plt.figure(figsize=(16, 24))

plt.subplot(3, 2, 1)
plt.imshow(probes[0], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 2)
plt.imshow(-probes[0], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 3)
plt.imshow(probes[1], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 4)
plt.imshow(-probes[1], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 5)
plt.imshow(probes[2], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 6)
plt.imshow(-probes[2], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

### Save for Asterix

In [ ]:
for i, probe in enumerate(probes):
    path = '/Users/ilaginja/Documents/LESIA/THD/Roman_probes/sinc_probes'
    fname = f'probe_{i}_sinc-freq1-{sinc_freq1}_sinc-freq2-{sinc_freq2}_sine-freq{sine_freq}_cgi-like.fits'
    fits.writeto(os.path.join(path, fname), probe)

## d) Sharp sincs

Big squares in the focal plane and undersampled functions in the pupil plane.

In [ ]:
act = 1 / 32
shift_x = -6 * act
shift_y = -7 * act

acts_across = 32
sinc_freq = 48   # cycles per DM
thetas = [2 * np.pi, 4 * np.pi / 3, 2 * np.pi / 3]
amp_ptv = 1   # nm

xvals_sinc = (np.arange(acts_across) / acts_across - 0.5 - shift_x) * sinc_freq
yvals_sinc = (np.arange(acts_across) / acts_across - 0.5 - shift_y) * sinc_freq
xx, yy = np.meshgrid(xvals_sinc, yvals_sinc)

probes = []
probe1 = amp_ptv * np.sinc(xx) * np.sinc(yy)
probe2 = amp_ptv * np.sinc(xx) * np.sinc(yy + np.pi / 2)
probe3 = amp_ptv * np.sinc(xx + np.pi / 2) * np.sinc(yy)
probes.append(probe1)
probes.append(probe2)
probes.append(probe3)

zerocenter = TwoSlopeNorm(vcenter=0.)

plt.figure(figsize=(16, 24))

plt.subplot(3, 2, 1)
plt.imshow(probes[0], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 2)
plt.imshow(-probes[0], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 3)
plt.imshow(probes[1], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 4)
plt.imshow(-probes[1], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 5)
plt.imshow(probes[2], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

plt.subplot(3, 2, 6)
plt.imshow(-probes[2], origin='lower', cmap='RdBu', norm=zerocenter)
plt.colorbar()

### Save for Asterix

In [ ]:
for i, probe in enumerate(probes):
    path = '/Users/ilaginja/Documents/LESIA/THD/Roman_probes/sinc_probes'
    fname = f'probe_{i}_sinc-freq-{sinc_freq}.fits'
    fits.writeto(os.path.join(path, fname), probe)

# 5. Plot with respect to the rest of the optical system as represented by Asterix

This takes any probes defined in Sec. 4 and runs them throught the Asterix optical model, convolves them by the influence function and plots them in the pupil plane.

In [ ]:
system_probe = []
for probe in range(len(probes)):
    dm_array = thd2.dm2.DM_pushact.copy()
    for act in range(thd2.dm2.DM_pushact.shape[0]):
        dm_array[act] = dm_array[act] / np.max(dm_array[act]) * probes[probe].ravel()[act]
    system_probe.append(np.sum(dm_array, axis=0))


zerocenter = TwoSlopeNorm(vcenter=0.)

plt.figure(figsize=(16, 24))

cmap_mask = cm.Greys.copy()
cmap_mask.set_bad(color='black')

for i in range(6):
    ax = plt.subplot(3, 2, i + 1)
    
    if i % 2 == 0:
        plt.imshow(system_probe[i // 2], origin='lower', cmap='RdBu', norm=zerocenter)
    else:
        plt.imshow(-system_probe[i // 2], origin='lower', cmap='RdBu', norm=zerocenter)
    plt.colorbar()
    plt.imshow(ls, origin='lower', cmap='Greys_r', alpha=0.3)
    #plt.imshow(masked_ls, origin='lower')#, cmap=cmap_mask)

#plt.savefig('/Users/ilaginja/probes_in_system.pdf')

Here I am trying to create a prettier plot with proper masking.

In [ ]:
plt.figure(figsize=(8, 8))

cmap_mask = cm.RdBu.copy()
cmap_mask.set_bad(color='black')

zerocenter = TwoSlopeNorm(vcenter=0., vmin=-0.7, vmax=0.7)

bbox = 50
masked_probe = np.ma.masked_where(masked_ls.mask, system_probe[2])

#plt.imshow(system_probe[0][bbox:-bbox, bbox:-bbox], origin='lower', cmap='RdBu', norm=zerocenter)
plt.imshow(masked_probe[bbox:-bbox, bbox:-bbox], origin='lower', cmap=cmap_mask, norm=zerocenter)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=20)
plt.xticks([])  # Remove x-axis ticks
plt.yticks([])  # Remove y-axis ticks

Plotting the probe command on top of a DM actuator grid (grids are not fully accurate here).

In [ ]:
# Add numbers on top of some pixels
#plt.text(33, 0, "0", ha='center', va='center', color="black", fontsize=8)

def apply_grid(ax):
    """Apply grid styling to a given axis."""
    ax.grid(which="minor", color='black', linestyle='-', linewidth=0.5)
    ax.set_xticks(np.arange(-0.5, 32, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, 32, 1), minor=True)
    ax.tick_params(which="both", bottom=False, left=False, labelbottom=False, labelleft=False)

    
zerocenter = TwoSlopeNorm(vcenter=0.)

plt.figure(figsize=(16, 24))

cmap_mask = cm.Greys.copy()
cmap_mask.set_bad(color='black')

# Plotting all subplots
for i in range(6):
    ax = plt.subplot(3, 2, i + 1)
    if i % 2 == 0:
        plt.imshow(probes[i // 2], origin='lower', cmap='RdBu', norm=zerocenter)
    else:
        plt.imshow(-probes[i // 2], origin='lower', cmap='RdBu', norm=zerocenter)
    plt.colorbar()
    apply_grid(ax)  # Apply grid to each subplot

#plt.savefig('probes_on_grid.pdf')

## Probe E-field in image plane

Looking at the pure FT of the probe in the image plane.

In [ ]:
probe = probes[2]

transform = np.fft.ifftshift(np.fft.fft2(np.fft.fftshift(probe)))

In [ ]:
imnorm = TwoSlopeNorm(vmin=-23, vcenter=0, vmax=20)

plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.imshow(np.real(transform), origin='lower', cmap='RdBu', norm=imnorm)
plt.title('Real part of probe transform')
plt.colorbar()
plt.subplot(2, 2, 2)
plt.imshow(np.imag(transform), origin='lower', cmap='RdBu')#, norm=imnorm)
plt.title('Imag part of probe transform')
plt.colorbar()
plt.subplot(2, 2, 3)
plt.imshow(np.abs(transform), origin='lower', cmap='Reds')#, norm=imnorm)
plt.title('Amplitude of probe transform')
plt.colorbar()
plt.subplot(2, 2, 4)
plt.imshow(np.angle(transform), origin='lower', cmap='RdBu')#, norm=imnorm)
plt.title('Phase of probe transform')
plt.colorbar()